In [1]:
!pip install textblob pandas


In [1]:
import pandas as pd

In [2]:
df_tweets = pd.read_csv(r"C:\Users\ayushi\Downloads\archive (25)\stock_tweets.csv")
df_tweets


,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."
...,...,...,...,...
80788,2021-10-07 17:11:57+00:00,Some of the fastest growing tech stocks on the...,XPEV,XPeng Inc.
80789,2021-10-04 17:05:59+00:00,"With earnings on the horizon, here is a quick ...",XPEV,XPeng Inc.
80790,2021-10-01 04:43:41+00:00,Our record delivery results are a testimony of...,XPEV,XPeng Inc.
80791,2021-10-01 00:03:32+00:00,"We delivered 10,412 Smart EVs in Sep 2021, rea...",XPEV,XPeng Inc.


In [3]:
from textblob import TextBlob

df_tweets.dropna(subset=["Tweet"], inplace=True)

# Clean tweets 
def clean_tweet(text):
    return ' '.join(word for word in text.split() if not word.startswith('http'))

df_tweets["Cleaned_Tweet"] = df_tweets["Tweet"].apply(clean_tweet)

# Generate sentiment score 
def get_sentiment_score(text):
    return TextBlob(text).sentiment.polarity  # returns a float between -1 (neg) to +1 (pos)

df_tweets["Sentiment_Score"] = df_tweets["Cleaned_Tweet"].apply(get_sentiment_score)

# Add sentiment label
def get_label(score):
    if score > 0:
        return "Positive"
    elif score < 0:
        return "Negative"
    else:
        return "Neutral"

df_tweets["Sentiment_Label"] = df_tweets["Sentiment_Score"].apply(get_label) 


In [4]:
df_stocks = pd.read_csv(r"C:\Users\ayushi\Downloads\stock_yfinance_data11 (1).csv")
df_stocks

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA
...,...,...,...,...,...,...,...,...
6295,2022-09-23,13.090000,13.892000,12.860000,13.710000,13.710000,28279600,XPEV
6296,2022-09-26,14.280000,14.830000,14.070000,14.370000,14.370000,27891300,XPEV
6297,2022-09-27,14.580000,14.800000,13.580000,13.710000,13.710000,21160800,XPEV
6298,2022-09-28,13.050000,13.421000,12.690000,13.330000,13.330000,31799400,XPEV


In [6]:


# Convert to datetime properly
df_tweets["Date"] = pd.to_datetime(df_tweets["Date"], errors='coerce')

df_tweets =df_tweets.dropna(subset=["Date"])
df_tweets["Date"] = df_tweets["Date"].dt.date

In [8]:

# Convert Date to datetime format
df_stocks["Date"] = pd.to_datetime(df_stocks["Date"], errors='coerce')
df_stocks = df_stocks.dropna(subset=["Date"])

# Extract only date part
df_stocks["Date"] = df_stocks["Date"].dt.date

In [9]:
# Aggregate sentiment
agg_sentiment = df_tweets.groupby(["Stock Name", "Date"]).agg({
    "Sentiment_Score": "mean",
    "Sentiment_Label": lambda x: x.mode()[0],  # most common label
    "Tweet": "count"
}).rename(columns={"Tweet": "Tweet_Count"}).reset_index()

In [15]:
merged = pd.merge(df_stocks, agg_sentiment, on=["Stock Name", "Date"], how="inner")

In [17]:
merged 

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name,Sentiment_Score,Sentiment_Label,Tweet_Count
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA,0.137768,Positive,90
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA,0.124179,Positive,94
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA,0.097135,Positive,119
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA,0.106853,Positive,88
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA,0.164837,Positive,78
...,...,...,...,...,...,...,...,...,...,...,...
4179,2022-08-30,18.280001,18.600000,17.950001,18.260000,18.260000,14484300,XPEV,0.100000,Positive,1
4180,2022-09-07,15.750000,16.530001,15.630000,16.000000,16.000000,14641500,XPEV,0.000000,Neutral,1
4181,2022-09-12,16.549999,16.639999,15.980000,16.360001,16.360001,21170100,XPEV,0.066667,Positive,1
4182,2022-09-19,14.300000,15.725000,14.290000,15.640000,15.640000,18193100,XPEV,0.000000,Neutral,1


In [25]:
merged.to_csv("merged_sentiment_stock_data.csv", index=False)
print(" Data merged successfully. Ready for visualization!")

 Data merged successfully. Ready for visualization!


In [27]:
merged.to_csv("C:/Users/ayushi/Downloads/merged_stock_sentiment_data.csv", index=False)
print(" Data merged successfully. Ready for visualization!")


 Data merged successfully. Ready for visualization!
